In [309]:
from utils_filter import *
from utils_datetime import *
from dateutil import parser
from dateutil.relativedelta import relativedelta
from geopy.geocoders import Nominatim


# TODO: 201804020000's day 3 forecasts are issued on 04/30--in reality, these forecasts were probably meant for 05/02. todo is clean for this sort of thing

Subsets outlooks, pph, and report data in various ways (add label to each datapoint for each method of subsetting so can be pulled out of full dataset later)


In [80]:
data_location = 'data'
moderate = False # only consider moderate days
outlooks, pph, reports = read_datasets(data_location, moderate)

reading outlooks 1
reading outlooks 2


c:\Users\miles\OneDrive\Documents\UW\Research\utils_filter.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  outlooks = outlooks.append(gp.read_file(data_location + '/outlooks/' + mod_string + '_outlooks_2.shp'))


reading pph
reading storm reports


In [85]:
tz_conversions = {'PST': timedelta(hours=8),
                  'MST': timedelta(hours=7),
                  'CST': timedelta(hours=6),
                  'CSt': timedelta(hours=6),
                  'CSC': timedelta(hours=6),
                  'SCT': timedelta(hours=6),
                  'EST': timedelta(hours=5),
                  'ESt': timedelta(hours=5),
                  'PDT': timedelta(hours=7),
                  'MDT': timedelta(hours=6),
                  'CDT': timedelta(hours=5),
                  'EDT': timedelta(hours=4),
                  'HST': timedelta(hours=10),
                  'SST': timedelta(hours=11),
                  'GST': timedelta(hours=10),
                  'AKS': timedelta(hours=9),
                  'AST': timedelta(hours=4),
                  'UNK': timedelta(hours=5),
                  'GMT': timedelta(0)}

def get_reports_date_strings(date_times, timezones):
    # returns list of strings of date of given datetime and timezone (where day cutoffs are 12-12 UTC) formatted as 'YYYYMMDD0000'
    for datetime, timezone, i in zip(date_times, timezones, range(len(timezones))):
        #print(datetime + ' ' + timezone[:3])
        datetime = parser.parse(datetime)
        datetime = datetime + tz_conversions[timezone[:3]]
        #print(datetime)
        if (datetime.hour < 12):
            datetime = datetime - timedelta(days = 1)
        if datetime.year > 2049:
            datetime = datetime - relativedelta(years = 100)
        datetime = datetime.strftime("%Y%m%d") + '0000'
        if i == 0:
            ret = [datetime]
        else:
            ret.append(datetime)
    return ret

def get_pph_date_strings(times):
    # returns a list of strings of given dates formatted as 'YYYYMMDD0000'
    for datetime, i in zip(times, range(len(times))):
        string = datetime.dt.strftime("%Y%m%d").values + '0000'
        if i == 0:
            ret = [string]
        else:
            ret.append(string)
    return ret

In [86]:
# add dates to reports and pph in same format as in outlooks
reports['DATE'] = get_reports_date_strings(reports['BEGIN_DATE_TIME'], reports['CZ_TIMEZONE']) 
pph['time'] = get_pph_date_strings(pph.time) 

In [87]:
# subset outlooks into only one day 1, two day 2, and one day 3 categorical outlooks 
# day 3: cycle not -1. day 2: cycle not -1. Day 1: cycle 6. Category: categorical or none.
outlooks = outlooks[(((outlooks['DAY'] == 1) & (outlooks['CYCLE'] == 6)) | ((outlooks['DAY'] == 2) & (outlooks['CYCLE'] != -1)) | ((outlooks['DAY'] == 3) & (outlooks['CYCLE'] != -1)))
         & (outlooks['CATEGORY'] != 'WIND') & (outlooks['CATEGORY'] != 'HAIL') & (outlooks['CATEGORY'] != 'TORNADO')& (outlooks['CATEGORY'] != 'ANY SEVERE')& (outlooks['CATEGORY'] != 'PROB')]

# reset incicies
outlooks = outlooks.reset_index(drop=True)

In [88]:
def add_outlooks_label(outlooks, label_dates, labels, label_name, none_label):
    # adds new column with values from labels on the corresponding list of dates in label_dates (DONE)
    print("adding a new column in outlooks")
    outlooks[label_name] = none_label
    for label, dates in zip(labels, label_dates):
        #print(label)
        outlooks[label_name].loc[outlooks['DATE'].isin(dates)] = label
    return outlooks

def add_pph_label(pph, label_dates, labels, label_name, none_label):
    # adds new variable with values from labels on the corresponding list of dates in label_dates (DONE)
    print("adding new variable in pph")
    pph[label_name] = (('time'), np.full(len(pph['time']), none_label))
    for label, dates in zip(labels, label_dates):
        #print(label)
        pph[label_name].loc[pph['time'].isin(dates)] = label  
    return pph

def add_reports_label(reports, label_dates, labels, label_name, none_label):
    # adds new column with values from labels on the corresponding list of dates in label_dates (DONE)
    reports[label_name] = none_label
    print("adding a new column in reports")
    for label, dates in zip(labels, label_dates):
       #print(label)
       reports[label_name].loc[reports['DATE'].isin(dates)] = label #
    return reports

def add_labels(outlooks, pph, reports, label_dates, labels, label_name, none_label):
    # adds labels, overwriting with later ones if a date has multiple labels
    return(add_outlooks_label(outlooks, label_dates, labels, label_name, none_label), 
           add_pph_label(pph, label_dates, labels, label_name, none_label),
           add_reports_label(reports, label_dates, labels, label_name, none_label))



Subset by threshold

In [89]:
# add max threshold forecasted for valid day to each datapoint

categories = ['TSTM', 'MRGL', 'SLGT', 'ENH', 'MDT', 'HIGH']
category_dates = []
for category in categories:
    category_dates.append(identify_dates_above_threshold(outlooks, category))

(new_outlooks, new_pph, new_reports) = add_labels(outlooks, pph, reports, category_dates, categories, 'MAX_CAT', 'NONE')



adding a new column in outlooks
adding new variable in pph


C:\Users\miles\AppData\Local\Temp\ipykernel_29832\1296049564.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  outlooks[label_name].loc[outlooks['DATE'].isin(dates)] = label


adding a new column in reports


C:\Users\miles\AppData\Local\Temp\ipykernel_29832\1296049564.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reports[label_name].loc[reports['DATE'].isin(dates)] = label #


Subset by ramp up/down amount

In [181]:
# dictionary of category to number
category_dict = {
    None : -1,
    'NONE' : -1,
    'TSTM': 0,
    'MRGL': 1,
    'SLGT': 2,
    'ENH': 3,
    'MDT': 4,
    'HIGH': 5
}

# put new_outlooks in correct order

new_outlooks['DATE_ORDER'] = 0
for index, row in new_outlooks.iterrows():
    if row['DAY'] == 3:
        new_outlooks.at[index, 'DATE_ORDER'] = row['DATE'] + '1'
    elif row['DAY'] == 1:
        new_outlooks.at[index, 'DATE_ORDER'] = row['DATE'] + '4'
    elif row['CYCLE'] == 7:
        new_outlooks.at[index, 'DATE_ORDER'] = row['DATE'] + '2'
    else:
        new_outlooks.at[index, 'DATE_ORDER'] = row['DATE'] + '3'
new_outlooks = new_outlooks.sort_values('DATE_ORDER')

ramp_ups = {
    0: [],
    1: [],
    2: [],
    3: [],
    4: [],
    5: [],
    6: []
}

ramp_downs = {
    0: [],
    -1: [],
    -2: [],
    -3: [],
    -4: [],
    -5: [],
    -6: []
}

In [182]:
# define and add ramp category for each datapoint. Potentially add 2 binary ramp up and ramp down (4 options are [up, down, up and down, niether]). How many forecasts to consider for each day? The day 3, both day 2, and the first day 1 (so 4 forecasts ramp)
old_date = '0'
old_do = '0'
first = True


for index, row in new_outlooks.iterrows(): #iterrating through each polygon in the outlook dataset
    cat = category_dict[row['THRESHOLD']]
    do = row['DATE_ORDER']
    date = row['DATE']

    if date != old_date: # New date, save ramp up and ramp down and save alongside old date, then reset ramps, max and min categories seen, and do threshold
        
        
        if first == True:
            first = False
        else:
            ramp_ups[ramp_up].append(old_date)
            ramp_downs[ramp_down].append(old_date)

        old_date = date
        old_do = do
        
        ramp_down = 0
        ramp_up = 0
        max_cat_date = -1
        
        if do[-1] == '1': 
            min_cat_date = 5 
        else: # First outlook for this date is not day 3, so day 3 had no outlook.
            min_cat_date = -1

        max_cat_do = cat


    elif do != old_do: # new outlook, update min and max categories seen, ramp value
        if max_cat_do - min_cat_date > ramp_up:
            ramp_up = max_cat_do - min_cat_date
        if max_cat_do - max_cat_date < ramp_down:
            ramp_down = max_cat_do - max_cat_date

        if max_cat_do > max_cat_date:
            max_cat_date = max_cat_do
        if max_cat_do < min_cat_date:
            min_cat_date = max_cat_do

        old_do = do

        max_cat_do = cat

    else: # Just another threshold within the same polygon
        if cat > max_cat_do:
            max_cat_do = cat
    

ramp_ups[ramp_up].append(old_date)
ramp_downs[ramp_down].append(old_date)

In [183]:
# ramp up
(new_outlooks, new_pph, new_reports) = add_labels(new_outlooks, new_pph, new_reports, list(ramp_ups.values()), ramp_ups.keys(), 'RAMP_UP', 'NONE')

# ramp down
(new_outlooks, new_pph, new_reports) = add_labels(new_outlooks, new_pph, new_reports, list(ramp_downs.values()), ramp_downs.keys(), 'RAMP_DOWN', 'NONE')


adding a new column in outlooks
adding new variable in pph
adding a new column in reports


C:\Users\miles\AppData\Local\Temp\ipykernel_29832\1296049564.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  outlooks[label_name].loc[outlooks['DATE'].isin(dates)] = label
C:\Users\miles\AppData\Local\Temp\ipykernel_29832\1296049564.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reports[label_name].loc[reports['DATE'].isin(dates)] = label #


adding a new column in outlooks
adding new variable in pph
adding a new column in reports


,ISSUE,EXPIRE,PRODISS,TYPE,DAY,THRESHOLD,CATEGORY,CYCLE,DATE,geometry,MAX_CAT,DATE_ORDER,RAMP_UP,RAMP_DOWN
0,198701091200,198701101200,198701080719,C,2,None,None,7,198701090000,None,NONE,1987010900002,0,0
1,198701091200,198701101200,198701081731,C,2,None,None,17,198701090000,None,NONE,1987010900003,0,0
2,198701161200,198701171200,198701160615,C,1,TSTM,CATEGORICAL,6,198701160000,"POLYGON ((-81.10300 30.60200, -81.08700 30.521...",TSTM,1987011600004,0,0
3,198701191200,198701201200,198701180703,C,2,None,None,7,198701190000,None,NONE,1987011900002,0,0
4,198701221200,198701231200,198701210800,C,2,None,None,7,198701220000,None,NONE,1987012200002,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73062,202312311200,202401011200,202312310520,C,1,None,None,6,202312310000,None,TSTM,2023123100004,0,-1
73060,202401011200,202401021200,202312300816,C,3,None,None,8,202401010000,None,NONE,2024010100001,0,0
73063,202401011200,202401021200,202312310600,C,2,None,None,7,202401010000,None,NONE,2024010100002,0,0
73065,202401011200,202401021200,202312311712,C,2,None,None,17,202401010000,None,NONE,2024010100003,0,0


Subset by accurate/inaccurate

Consider forecast issue time for day 1--subset reports (and revise pph?)--look at other studies to see how they handled multiple day 1 outlooks (see the one in slack). Starting point should be use first forecasts, later ones would be a different analysis for another time. Decide if we should denote these somehow..

Subset by season

In [218]:
# add column denoting season (4 met seasons as starting point)
dates = new_outlooks['DATE'].unique()
seasons = ['Winter', 'Spring', 'Summer', 'Fall']
season_dates = [[], [], [], []]
for date in dates:
    month = int(date[4:6])
    if month == 12 or month < 3:
        season_dates[0].append(date)
    elif month < 6:
        season_dates[1].append(date)
    elif month < 9:
        season_dates[2].append(date)
    else:
        season_dates[3].append(date)

(new_outlooks, new_pph, new_reports) = add_labels(new_outlooks, new_pph, new_reports, season_dates, seasons, 'SEASON', 'NONE')


adding a new column in outlooks
adding new variable in pph
adding a new column in reports


C:\Users\miles\AppData\Local\Temp\ipykernel_29832\1296049564.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  outlooks[label_name].loc[outlooks['DATE'].isin(dates)] = label
C:\Users\miles\AppData\Local\Temp\ipykernel_29832\1296049564.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reports[label_name].loc[reports['DATE'].isin(dates)] = label #


Subset by region

In [242]:
# decide how to do... lower priotity, could do by state of center of highest category (or center of PPH?) Could start with super basic version
# same regions as Anderson-Frey 2016
# center is grid square with highest p_perfect prob of at least one hazard (assuming each hazard in independent)
new_pph['p_perfect_total'] = (1 - (1-new_pph['p_perfect_wind']/100)*(1-new_pph['p_perfect_hail']/100)*(1-new_pph['p_perfect_tor']/100))*100


In [349]:
geolocator = Nominatim(user_agent="severe_thunderstorm_miles")
west_threshold_co_nm = -105.5
regions_dict = { # list of states fully within each region (doesn't include AK, HI, CO, NM)
    'West': ['Washington', 'Oregon', 'California', 'Idaho', 'Montana', 'Wyoming', 'Utah', 'Arizona'],
    'Midwest': ['North Dakota', 'South Dakota', 'Minnesota', 'Iowa', 'Wisconsin', 'Illinois', 'Michigan', 'Indiana', 'Ohio', 'Kentucky'],
    'Great Plains': ['Nebraska', 'Kansas', 'Oklahoma', 'Texas', 'Missouri'],
    'Northeast': ['Maine', 'Vermont', 'New Hampshire', 'Massachusetts', 'Rhode Island', 'Connecticut', 'New York', 'Pennsylvania', 'New Jersey', 'Delaware', 'Maryland', 'District of Columbia', 'West Virginia'],
    'South': ['Virginia', 'Arkansas', 'Louisiana', 'Tennessee', 'Mississippi', 'Alabama', 'Georgia', 'North Carolina', 'South Carolina', 'Florida']
}


def get_state(lat, lon):
    location = geolocator.reverse(str(lat)+","+str(lon))
    if location == None:
        return None
    address = location.raw['address']
    state = address.get('state', '')
    return state

def get_region(lat, lon):
    state = get_state(lat, lon)
    if state == 'Colorado' or state == 'New Mexico':
        if lon < west_threshold_co_nm:
            return('West')
        else:
            return('Great Plains')
    for region in regions_dict:
        if state in regions_dict[region]:
            return region
    return('NONE')

regions = {
    'West': [],
    'Midwest': [],
    'Great Plains': [],
    'Northeast': [],
    'South': [],
    'NONE': []
}

In [350]:
old_year = ''
for date, date_pph in new_pph.groupby('time'):
    if date_pph['p_perfect_total'].max() > 0:
        year = date[0:3]
        if year != old_year:
            print(year)
            old_year = year
        max_coords = date_pph['p_perfect_total'].argmax(dim = ['x', 'y'])
        max_x_coord = max_coords['x'].values
        max_y_coord = max_coords['y'].values
        lat = date_pph['lat'].loc[dict(x = max_x_coord, y = max_y_coord)].values
        lon = date_pph['lon'].loc[dict(x = max_x_coord, y = max_y_coord)].values
        region = get_region(lat, lon)
        regions[region].append(date)
regions



197901010000
Alabama
South
197901020000
Florida
South
197901120000
None
NONE
197901180000

NONE
197901190000
Texas
Great Plains
197901200000
Florida
South
197901230000
Florida
South
197901310000
California
West
197902220000
Texas
Great Plains
197902230000
Texas
Great Plains
197902240000
Alabama
South
197902250000
North Carolina
South
197902270000
Texas
Great Plains
197902280000
Mississippi
South
197903020000
Texas
Great Plains
197903030000
Florida
South
197903070000
South Dakota
Midwest
197903140000
California
West
197903150000
Texas
Great Plains
197903170000
Texas
Great Plains
197903180000
Kansas
Great Plains
197903190000
Texas
Great Plains
197903200000
Texas
Great Plains
197903210000
Texas
Great Plains
197903220000
Kentucky
Midwest
197903230000
Georgia
South
197903240000
Virginia
South
197903260000
Texas
Great Plains
197903280000
Arizona
West
197903290000
Texas
Great Plains
197903300000
Louisiana
South
197903310000
Oklahoma
Great Plains
197904010000
Texas
Great Plains
197904020000
Te

In [ ]:
(new_outlooks, new_pph, new_reports) = add_labels(new_outlooks, new_pph, new_reports, list(regions.values()), regions.keys(), 'REGION', 'NONE')


Subset by environmental data (to do later)

In [185]:
# resave labelled data
outlook_save_location = 'data/outlooks'
pph_save_location = 'data/pph'
report_save_location = 'data/storm_reports'

new_outlooks.to_file(outlook_save_location + '/labelled_outlooks.shp')
new_pph.to_netcdf(pph_save_location + '/labelled_pph.nc')
new_reports.to_csv(report_save_location + '/labelled_reports.csv')

In [87]:
# TODO: create master function that subsets data based on easy inputs once it's all been labelled. Put in a util file
new_outlooks

,ISSUE,EXPIRE,PRODISS,TYPE,DAY,THRESHOLD,CATEGORY,CYCLE,DATE,geometry,MAX_CAT
36,198701091200,198701101200,198701080719,C,2,None,None,7,198701090000,None,NONE
38,198701091200,198701101200,198701081731,C,2,None,None,17,198701090000,None,NONE
73,198701161200,198701171200,198701160615,C,1,TSTM,CATEGORICAL,6,198701160000,"POLYGON ((-81.10300 30.60200, -81.08700 30.521...",TSTM
85,198701191200,198701201200,198701180703,C,2,None,None,7,198701190000,None,NONE
104,198701221200,198701231200,198701210800,C,2,None,None,7,198701220000,None,NONE
...,...,...,...,...,...,...,...,...,...,...,...
183175,202312311200,202401011200,202312301607,C,2,None,None,17,202312310000,None,TSTM
183178,202312311200,202401011200,202312310520,C,1,None,None,6,202312310000,None,TSTM
183179,202401011200,202401021200,202312310600,C,2,None,None,7,202401010000,None,NONE
183180,202401021200,202401031200,202312310815,C,3,TSTM,CATEGORICAL,8,202401020000,"POLYGON ((-90.74500 28.71100, -91.08300 28.732...",TSTM
